In [2]:
Folder1 = "test/txt"
Folder2 = "train/beth/txt"
Folder3 = "train/partners/txt"

In [90]:
import os
import re
import time
import requests
import pandas as pd
import ast
import json

In [6]:
BlackList = ["YES", "NO", "NORMAL", "REPORT", "OTHER", "SIZE", "TIME", "DATE", "DOCTOR", "TIME", "PLAN", "FOLLOW", "MEDICINE", "REVIEW", "COUNT"]

# Functions -start-

In [7]:
def GetAllFilesListInFolder(f):
    files = os.listdir(f)
    textFiles = []
    
    for file in files:
        if file.endswith(".txt"):
            textFiles.append(os.path.join(f, file))
    return textFiles


def ReadAllFilesInList(Files):
    content = []
    for file in Files:
        with open(file, "r", encoding='utf-8-sig') as f:
            content.append(f.read())
    return content

def FindAbbreviations(text):
    # (?=[A-Z0-9]*[A-Z])
    # İfadenin devamında sıfır veya daha fazla "büyük harf ve rakam" ve ardından en az bir büyük harf olmalıdır.
    #
    # [A-Z0-9]{2,}
    # En az bir büyük harf ve rakamdan oluşan, en az 2 karakterlik veri
    #
    # Kombinasyonlu ifade de bu ifadelerin arasına "/" karakteri gelirse de kabul ediyor.
    # DM2, PCRE, DM2/A (kombinasyonlu)
    abbreviation_pattern = r'\b(?=[A-Z0-9]*[A-Z])[A-Z0-9]{2,6}\b'
    combined_pattern = r'\b(?=[A-Z0-9]*[A-Z])[A-Z0-9]{2,6}/(?=[A-Z0-9]*[A-Z])[A-Z0-9]{1,5}\b'

    tokens = re.findall(abbreviation_pattern, text)
    tokens.extend(re.findall(combined_pattern, text))

    abbreviations = {token for token in tokens if re.search(r'[A-Z]', token) and token not in BlackList and not token[0].isdigit()}

    return sorted(abbreviations)

def FilterAbbreviationsWithTreshold(allAbbreviations, threshold):
    # Çok fazla kısaltma bulunurken, bunların bazıları yanlış olabilir.
    # Az sayıda tekrar eden veri varsa bunları sileceğiz, belirli bir eşik değerin üstündeki
    # tekrar sayıları o verinin potansiyel kısaltma olduğunu ifade edeceğini kabul ediyoruz.
    abbreviationsCount = {}

    for abbreviation in allAbbreviations:
        abbreviationsCount[abbreviation] = abbreviationsCount.get(abbreviation, 0) + 1

    return [abbr for abbr, count in abbreviationsCount.items() if count > threshold]

# Functions -end-

In [8]:
Files1 = GetAllFilesListInFolder(Folder1)
Files2 = GetAllFilesListInFolder(Folder2)
Files3 = GetAllFilesListInFolder(Folder3)

Content1 = ReadAllFilesInList(Files1)
Content2 = ReadAllFilesInList(Files2)
Content3 = ReadAllFilesInList(Files3)

In [9]:
AllAbbreviations = []

for Contents in [Content1, Content2, Content3]:
    for Content in Contents:
        arr = FindAbbreviations(Content)
        AllAbbreviations.extend(arr)

AllAbbreviations = FilterAbbreviationsWithTreshold(AllAbbreviations, 3)
AllAbbreviations = set(AllAbbreviations)
print(AllAbbreviations)

{'HPI', 'PCA', 'AT', 'JOB', 'ET', 'CENTER', 'UTERUS', 'PND', 'WED', 'UTI', 'LVH', 'CPAP', 'TP', 'BACK', 'KCL', 'IN', 'CMO', 'END', 'POD', 'IVC', 'CCU', 'MAE', 'ANEMIA', 'LIPASE', 'HGB', 'SIGNS', 'NODES', 'MD', 'URI', 'GEN', 'AP', 'SUN', 'IVF', 'TOTAL', 'BLOOD', 'MM', 'FEV1', 'ARTERY', 'DH', 'FROM', 'LP', 'ANION', 'TAH', 'FIO2', 'HOLD', 'XS', 'BYPASS', 'VITS', 'RIGHT', 'XRT', 'TID', 'TR', 'IF', 'CVP', 'FEN', 'MUMPS', 'T11', 'P1', 'EH', 'L5', 'AVF', 'NVH', 'SR', 'S1', 'PUMC', 'NP', 'OK', 'UNIT', 'ZOCOR', 'NA', 'EXTREM', 'D10W', 'RESUME', 'PAF', 'PARKS', 'DO', 'USOH', 'HDL', 'TYPE', 'LDL', 'AV', 'UPON', 'FVC', 'UE', 'WBC', 'ANA', 'SC', 'MVR', 'PERRL', 'CAR', 'RR', 'NGTD', 'HS', 'AVR', 'UP', 'BID', 'GGI', 'RDW', 'MIBI', 'LENNI', 'T1', 'DAY', 'KETONE', 'TIBC', 'DM2', 'MARK', 'MR', 'NAD', 'NEURO', 'HEIGHT', 'NOT', 'DNR/DNI', 'MCH', 'MRN', 'COURSE', 'IM', 'ELMVH', 'D5', 'APPEAR', 'LI', 'MONOS', 'Q24H', 'Q12H', 'SODIUM', 'PCO2', 'SSS', 'RLL', 'HTN', 'CPR', 'FDI', 'MRA', 'GAP', 'ATRIAL', 'NGT',

In [10]:
TRFile = "correction_1900.xlsx"
TRDatas = pd.read_excel(TRFile, sheet_name=None)
try:
    TRDatas = TRDatas["Sheet1"]
except:
    pass

In [11]:
TRDatas

,Index,Content,Notes,Content.1
0,En(1),# AV fistula : pt with h/o numerous clots in A...,AV:arteriovenous:arteriyovenöz,# AV fistula : pt with h/o numerous clots in A...
1,Tr_second(1),# AV fistül: AV fistülde çok sayıda pıhtı öyk...,NaN,# AV fistül: AV fistülde çok sayıda pıhtı öyk...
2,Corrected(1),# AV fistül : AV fistülde çok sayıda pıhtı öyk...,NaN,NaN
3,NaN,NaN,NaN,NaN
4,En(2),The patient had coarse rhonchi on examination .,NaN,The patient had coarse rhonchi on examination .
...,...,...,...,...
7594,Corrected(1899),Agresif kan ürünü replasmanına rağmen hasta 12...,NaN,NaN
7595,NaN,NaN,NaN,NaN
7596,En(1900),He had undergone C-spine surgery for spinal st...,NaN,He had undergone C-spine surgery for spinal st...
7597,Tr_second(1900),Sol üst ekstremitede rezidüel spastik parezi v...,NaN,Sol üst ekstremitede rezidüel spastik parezi v...


In [12]:
TRDatas = TRDatas.dropna(subset=["Notes"])
TRDatas

,Index,Content,Notes,Content.1
0,En(1),# AV fistula : pt with h/o numerous clots in A...,AV:arteriovenous:arteriyovenöz,# AV fistula : pt with h/o numerous clots in A...
12,En(4),#. Pleural effusions : Likely due to CHF altho...,CHF: congestive heart failure:KKY: Konjestif k...,#. Pleural effusions : Likely due to CHF altho...
13,Tr_second(4),#. Plevral efüzyonlar : dış merkeze kabulünde ...,Admission kelimesinin hem hastaneye yatış hem ...,#. Plevral efüzyonlar : dış merkeze kabulünde ...
20,En(6),"Atrioseptal defect repair in 1970 , cardiomyop...",CPAP: Continuous Positive Airway Pressure: Dev...,"Atrioseptal defect repair in 1970 , cardiomyop..."
24,En(7),69 y/o female wtih PMH significant for ESRD on...,PMH:Past Medical History: özgeçmiş; ESRD: End-...,69 y/o female wtih PMH significant for ESRD on...
...,...,...,...,...
7338,Corrected(1835),"Azitromisin tedavisi sonlandırıldıktan sonra, ...",advers reaksiyon ve yan etki aynı anlamda kull...,NaN
7352,En(1839),He had been on a diazepam CIWA scale prophylax...,DTs: Delirium Tremens: Deliryum tremens.,He had been on a diazepam CIWA scale prophylax...
7392,En(1849),5) Anemia - previous w/u c/w ACD ( elev ferrit...,w/u: Workup: tetkikler; ACD: Anemia of chronic...,5) Anemia - previous w/u c/w ACD ( elev ferrit...
7512,En(1879),4. CAD s/p velocity stenting proximal LAD 2002,Velocity bir stent markası.,4. CAD s/p velocity stenting proximal LAD 2002


In [13]:
TRDatas = TRDatas.drop(columns=['Index', 'Content', 'Content.1'])
TRDatas

,Notes
0,AV:arteriovenous:arteriyovenöz
12,CHF: congestive heart failure:KKY: Konjestif k...
13,Admission kelimesinin hem hastaneye yatış hem ...
20,CPAP: Continuous Positive Airway Pressure: Dev...
24,PMH:Past Medical History: özgeçmiş; ESRD: End-...
...,...
7338,advers reaksiyon ve yan etki aynı anlamda kull...
7352,DTs: Delirium Tremens: Deliryum tremens.
7392,w/u: Workup: tetkikler; ACD: Anemia of chronic...
7512,Velocity bir stent markası.


In [13]:
TRDatas.to_json("TRDatas.json", orient="records" indent=4)

# Veri Düzenleme
Daha önceden hazırlanmış çeviri verilerinin standart sağlamak amacıyla şu şekilde düzenleme yapılmıştır:
```
EN_KISALTMA:EN_AÇIKLAMA:TR_AÇIKLAMA:TR_KISALTMA
```

EN_AÇIKLAMA ve TR_AÇIKLAMA ya string oluyor ya da çoğul anlamdaysa [TR_AÇIKLAMA_1, TR_AÇIKLAMA_2] olarak çoğaltılıyor.
```

Eksik olan veriler boş bırakılıyor, yeni veri varsa ; işaretiyle devam ediyor. Örnek:

```json
        "12":"CHF:congestive heart failure:Konjestif kalp yetmezliği:KKY|OSH:outside hospital:dış merkez",
        "13":"Admission::[hastaneye yatış, hastaneye başvurmak]",
```

In [29]:
TRDatas = pd.read_json("TRDatas_duzeltilmis.json", orient="records")

In [30]:
TRDatas

,Notes
0,AV:arteriovenous:arteriyovenöz
12,CHF:congestive heart failure:Konjestif kalp ye...
13,"Admission::[""hastaneye yatış"", ""hastaneye başv..."
20,CPAP:Continuous Positive Airway Pressure:Devam...
24,PMH:Past Medical History:özgeçmiş; ESRD:End-St...
...,...
7338,advers reaksiyon ve yan etki aynı anlamda kull...
7352,DTs: Delirium Tremens: Deliryum tremens.
7392,w/u: Workup: tetkikler; ACD: Anemia of chronic...
7512,


In [81]:
results = {}
for _, row in TRDatas.iterrows():
    note = row["Notes"].strip()
    
    if not note:
        continue  # Boş olanları atla

    # Kısaltma bloklarını ";" ile ayır
    blocks = note.split(";")

    for block in blocks:
        # Regex ile "EN_KISALTMA:EN_AÇIKLAMA:TR_AÇIKLAMA" desenini bul
        match = re.match(r"(.*?):\s*(.*?):\s*(.*)", block.strip())
        if match:
            en_kisaltma, en_aciklama, tr_aciklama = match.groups()
            tr_aciklama = tr_aciklama.strip()
            tr_aciklama = tr_aciklama.replace(":", "")

            if tr_aciklama.startswith("[") and tr_aciklama.endswith("]"):
                try:
                    tr_aciklama_list = ast.literal_eval(tr_aciklama)
                    if not isinstance(tr_aciklama_list, list):
                        tr_aciklama_list = [tr_aciklama_list]
                except:
                    tr_aciklama_list = tr_aciklama.strip()
            else:
                tr_aciklama_list = [x.strip() for x in tr_aciklama.split(",")] if "," in tr_aciklama else tr_aciklama.strip()

            print("list = " + str(tr_aciklama_list) + " " + str(type(tr_aciklama_list)))

            if type(tr_aciklama_list) != type([]):
                tr_aciklama_list = [tr_aciklama_list]

            for tr_aciklama_list_element in tr_aciklama_list:
                if en_kisaltma in results:
                    # Eğer aynı tr_aciklama_list_element varsa eklememesi için kontrol:
                    existing_tr_text = [entry["tr_text"] for entry in results[en_kisaltma]]
                    if tr_aciklama_list_element not in existing_tr_text:
                        results[en_kisaltma].append({
                            "en_text": en_aciklama,
                            "tr_text": tr_aciklama_list_element,
                            "source": "i2b2"
                        })
                else:
                    results[en_kisaltma] = [{
                        "en_text": en_aciklama,
                        "tr_text": tr_aciklama_list_element,
                        "source": "i2b2"
                    }]

list = arteriyovenöz <class 'str'>
list = Konjestif kalp yetmezliğiKKY <class 'str'>
list = dış merkez <class 'str'>
deneme
deneme2
list = ['hastaneye yatış', 'hastaneye başvurmak'] <class 'list'>
list = Devamlı Pozitif Havayolu Basıncı <class 'str'>
list = özgeçmiş <class 'str'>
list = Son Dönem Böbrek HastalıgıSDBH <class 'str'>
list = hemodiyaliz <class 'str'>
list = Tip 2 Diabetes MellitusTip 2 DM <class 'str'>
list = Pulmoner Emboli (Tromboembolizm)PTE <class 'str'>
list = nabızsız elektriksel aktiviteNEA <class 'str'>
list = ElektrokardiyogramEKG <class 'str'>
list = Tam idrar tahliliTiT <class 'str'>
list = idrar yolu enfeksiyonu iYE <class 'str'>
list = infüzyon <class 'str'>
list = Göğüs röntgeni  AC Grafi <class 'str'>
list = Sağ alt lob <class 'str'>
list = total parenteral beslenme <class 'str'>
deneme
deneme2
list = ['Nefes darlığı', 'dispne'] <class 'list'>
list = Bilgisayarlı Tomografi  BT <class 'str'>
list = Primer sklerozan kolanjit PSK <class 'str'>
list =  <class 's

In [82]:
results

{'AV': [{'en_text': 'arteriovenous',
   'tr_text': 'arteriyovenöz',
   'source': 'i2b2'}],
 'CHF': [{'en_text': 'congestive heart failure',
   'tr_text': 'Konjestif kalp yetmezliğiKKY',
   'source': 'i2b2'}],
 'OSH': [{'en_text': 'outside hospital',
   'tr_text': 'dış merkez',
   'source': 'i2b2'}],
 'Admission': [{'en_text': '', 'tr_text': 'hastaneye yatış', 'source': 'i2b2'},
  {'en_text': '', 'tr_text': 'hastaneye başvurmak', 'source': 'i2b2'}],
 'CPAP': [{'en_text': 'Continuous Positive Airway Pressure',
   'tr_text': 'Devamlı Pozitif Havayolu Basıncı',
   'source': 'i2b2'},
  {'en_text': 'Continuous positive airway pressure',
   'tr_text': 'Sürekli pozitif hava yolu basıncı',
   'source': 'i2b2'}],
 'PMH': [{'en_text': 'Past Medical History',
   'tr_text': 'özgeçmiş',
   'source': 'i2b2'}],
 'ESRD': [{'en_text': 'End-Stage Renal Disease',
   'tr_text': 'Son Dönem Böbrek HastalıgıSDBH',
   'source': 'i2b2'}],
 'HD': [{'en_text': 'hemodialysis',
   'tr_text': 'hemodiyaliz',
   'sour

In [87]:
len_total_desc = 0

for k in results.keys():
    k_len = len(results[k])
    len_total_desc = len_total_desc + k_len
    if k_len > 1:
        print(k + " => " + str(k_len))

Admission => 2
CPAP => 2
RLL => 3
TPN => 2
SOB => 2
IV => 2
ICU => 3
PO => 2
ASA => 3
GI => 2
LAD => 2
PCWP => 3
CMED => 2
SAB => 2
PNA => 2
CCU => 2
MMP => 2
BP => 2
NS => 2
SBP => 2
POD => 2
admission => 2
Cx => 3
RP => 3
RCA => 2
BID => 2
IVF => 2
TTE => 2
PM => 2
CABG => 2
EGD => 2
RUQ => 2
HK => 2
RA => 2
STEMI => 2
DDD => 2
VS => 2
MS => 2
UE => 2
LVEDP => 2
LVEF => 2
DO => 2
PCP => 2
HEENT => 4
CNS => 2
CEA => 2
CBC => 2
M,W, F => 3
KUB => 2
PERRL => 3
IDDM => 2


In [88]:
print("Toplam " + str(len(results.keys())) + " kısaltma ve bu kısaltmalara karşılık gelen toplam " + str(len_total_desc))

Toplam 347 kısaltma ve bu kısaltmalara karşılık gelen toplam 408


In [92]:
with open("Dictionary.json", "w", encoding="utf-8") as json_file:
    json.dump(results, json_file, ensure_ascii=False, indent=4)

In [93]:
results

{'AV': [{'en_text': 'arteriovenous',
   'tr_text': 'arteriyovenöz',
   'source': 'i2b2'}],
 'CHF': [{'en_text': 'congestive heart failure',
   'tr_text': 'Konjestif kalp yetmezliğiKKY',
   'source': 'i2b2'}],
 'OSH': [{'en_text': 'outside hospital',
   'tr_text': 'dış merkez',
   'source': 'i2b2'}],
 'Admission': [{'en_text': '', 'tr_text': 'hastaneye yatış', 'source': 'i2b2'},
  {'en_text': '', 'tr_text': 'hastaneye başvurmak', 'source': 'i2b2'}],
 'CPAP': [{'en_text': 'Continuous Positive Airway Pressure',
   'tr_text': 'Devamlı Pozitif Havayolu Basıncı',
   'source': 'i2b2'},
  {'en_text': 'Continuous positive airway pressure',
   'tr_text': 'Sürekli pozitif hava yolu basıncı',
   'source': 'i2b2'}],
 'PMH': [{'en_text': 'Past Medical History',
   'tr_text': 'özgeçmiş',
   'source': 'i2b2'}],
 'ESRD': [{'en_text': 'End-Stage Renal Disease',
   'tr_text': 'Son Dönem Böbrek HastalıgıSDBH',
   'source': 'i2b2'}],
 'HD': [{'en_text': 'hemodialysis',
   'tr_text': 'hemodiyaliz',
   'sour